In [3]:
import pandas as pd
import xgboost as xgb
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression


In [ ]:
# 1. Load and preprocess the dataset
file_path = 'stacking.csv'
df = pd.read_csv(file_path)

# Convert Date column to datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Split data into training (2021–2024) and testing (2025) sets
train_df = df[(df['Date'].dt.year >= 2021) & (df['Date'].dt.year <= 2024)]
test_df = df[df['Date'].dt.year == 2025]

# Define feature columns (base model predictions) and target column
feature_columns = ['KNN', 'LSTM','XG'] # choose the model
target_column = 'movement'

X_train = train_df[feature_columns]
y_train = train_df[target_column]
X_test = test_df[feature_columns]
y_test = test_df[target_column]

# 2. Evaluate accuracy of each base model
print("\nBase Models Accuracy:")
base_model_scores = {}
print(f"{'Model':<10} | {'Train Acc':<12} | {'Test Acc':<12}")
print("-"*40)
for col in feature_columns:
    # Convert predicted probabilities to binary labels
    train_pred = train_df[col].apply(lambda x: 1 if x >= 0.5 else 0)
    test_pred = test_df[col].apply(lambda x: 1 if x >= 0.5 else 0)
    
	# Compute accuracy on training and testing sets
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    
    base_model_scores[col] = test_acc
    print(f"{col:<10} | {train_acc:<12.4f} | {test_acc:<12.4f}")
print("-"*40)

# 3. Define hyperparameter grid for Grid Search
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [1, 2, 3],
    'learning_rate': [0.01, 0.02, 0.005]
}

# 4. Create XGBoost classifier and apply GridSearchCV
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=0
)

# Perform 3-fold cross-validation grid search
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',   
    cv=3,                
    verbose=1,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)
print("Best CV Accuracy:", grid_search.best_score_)

# Evaluate the best stacking model on test set
best_model = grid_search.best_estimator_

# extract predict class labels and probabilities
y_pred_stack = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# Compute test accuracy
stacking_acc = accuracy_score(y_test, y_pred_stack)
print(f"Stacking Accuracy on Test Set: {stacking_acc:.4f}")

# Display feature importance of base models
importance_df = pd.DataFrame({
    'Model': feature_columns,
    'Weight': best_model.feature_importances_
}).sort_values(by='Weight', ascending=False)
print("\nFeature Importance:\n", importance_df)


# Store original predicted probability
output_df = test_df[['Date']].copy()
output_df['Probability'] = y_pred_proba
output_df['Probability'] = (y_pred_proba - np.min(y_pred_proba)) / (np.max(y_pred_proba) - np.min(y_pred_proba))
output_df['Actual'] = y_test.values

output_df.to_csv('stacking_pred_prob.csv', index=False)


Base Models Accuracy:
Model      | Train Acc    | Test Acc    
----------------------------------------
KNN        | 0.5572       | 0.6505      
LSTM       | 0.5736       | 0.6262      
XG         | 0.5005       | 0.5631      
----------------------------------------
Fitting 3 folds for each of 27 candidates, totalling 81 fits
Best Parameters: {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100}
Best CV Accuracy: 0.6179082419192499
Stacking Accuracy on Test Set: 0.6796

Feature Importance:
   Model    Weight
0   KNN  0.527925
1  LSTM  0.259847
2    XG  0.212228
